In [1]:
import pandas as pd
import sys
from data_reader import fes_reader, perf_reader
from tqdm import tqdm
import bisect
import json
import datetime

In [2]:
# конвертация путей файлов в зависимости от системы
def replace_slash(file_path):
    platform = sys.platform
    slash_map = {'win32': '\\',
                'cygwin': '\\',
                'darwin': '/',
                'linux2': '/'}
    if platform not in slash_map.keys(): platform = 'linux2'
    return file_path.replace('\\', slash_map[platform])

In [3]:
perf_path = replace_slash('input_data\\' + 'armitz/miner/ARX_PERF.xlsx')
fes_path = replace_slash('input_data\\armitz/miner/QINF.xlsx')

In [4]:
perf_df = perf_reader(perf_path)
fes_df = fes_reader(fes_path)
SOIL_CUT = 60

started reading perf xl
done reading perf xl
started reading fes xl
done reading fes xl


In [8]:
fes_df.set_index('well', inplace=True)

fes_dict = fes_df.groupby(level=0, sort=False) \
    .apply(lambda x: [{'top': e[0],
                       'bot': e[1],
                       'soil': e[2]}
                      for e in x.values]) \
    .to_dict()

In [29]:
fes_dict


{'1': [{'top': 1669.6, 'bot': 1670.6, 'soil': 69.5},
  {'top': 1689.6, 'bot': 1692.4, 'soil': 91.7},
  {'top': 1699.6, 'bot': 1702.8, 'soil': 91.0},
  {'top': 1709.8, 'bot': 1712.0, 'soil': 77.0}],
 '1н': [{'top': 1583.6, 'bot': 1586.4, 'soil': 70.0},
  {'top': 1609.2, 'bot': 1611.6, 'soil': 81.9}],
 '2': [{'top': 1680.8, 'bot': 1683.2, 'soil': 94.0},
  {'top': 1684.2, 'bot': 1694.0, 'soil': 94.0}],
 '3н': [{'top': 1589.6, 'bot': 1591.0, 'soil': 78.6},
  {'top': 1615.4, 'bot': 1618.4, 'soil': 76.0}],
 '3п': [{'top': 1593.0, 'bot': 1594.6, 'soil': 74.1},
  {'top': 1631.2, 'bot': 1638.8, 'soil': 74.0}],
 '4н': [{'top': 1577.6, 'bot': 1579.0, 'soil': 78.6}],
 '8н': [{'top': 1760.4, 'bot': 1761.2, 'soil': 50.0},
  {'top': 1771.8, 'bot': 1772.4, 'soil': 71.1},
  {'top': 1780.8, 'bot': 1782.6, 'soil': 71.0},
  {'top': 1790.6, 'bot': 1792.8, 'soil': 81.9},
  {'top': 1792.8, 'bot': 1796.2, 'soil': 81.2},
  {'top': 1797.2, 'bot': 1802.8, 'soil': 64.1}],
 '9н': [{'top': 1770.6, 'bot': 1773.2, 's

In [ ]:
# проверка пласта на перфорированность
def is_perf(top, bot, ints):
    if ints is None:
        return False
    for int_perf in ints:
        if (top < int_perf['bot']) and (int_perf['top'] < bot):
            return True
    return False


def find_layers(perf_ints, fes_dict, soil_cut):
    lost_layers = []
    for well in tqdm(fes_dict.keys()):
        ints = perf_ints.get(well)
        for row in fes_dict[well]:
            top = row['top']
            bot = row['bot']
            soil = row['soil']
            if soil < soil_cut:
                continue
            if (not is_perf(top, bot, ints)):
                lost_layers.append({'well': well, 'top': top, 'bot': bot, 'soil': soil})
    return lost_layers

plasts = find_layers(perf_df, fes_dict, SOIL_CUT)

In [31]:
plasts

[{'well': '1', 'top': 1669.6, 'bot': 1670.6, 'soil': 69.5},
 {'well': '1', 'top': 1689.6, 'bot': 1692.4, 'soil': 91.7},
 {'well': '1', 'top': 1699.6, 'bot': 1702.8, 'soil': 91.0},
 {'well': '1', 'top': 1709.8, 'bot': 1712.0, 'soil': 77.0},
 {'well': '1н', 'top': 1583.6, 'bot': 1586.4, 'soil': 70.0},
 {'well': '1н', 'top': 1609.2, 'bot': 1611.6, 'soil': 81.9},
 {'well': '2', 'top': 1680.8, 'bot': 1683.2, 'soil': 94.0},
 {'well': '2', 'top': 1684.2, 'bot': 1694.0, 'soil': 94.0},
 {'well': '3н', 'top': 1589.6, 'bot': 1591.0, 'soil': 78.6},
 {'well': '3н', 'top': 1615.4, 'bot': 1618.4, 'soil': 76.0},
 {'well': '4н', 'top': 1577.6, 'bot': 1579.0, 'soil': 78.6},
 {'well': '8н', 'top': 1771.8, 'bot': 1772.4, 'soil': 71.1},
 {'well': '8н', 'top': 1780.8, 'bot': 1782.6, 'soil': 71.0},
 {'well': '8н', 'top': 1790.6, 'bot': 1792.8, 'soil': 81.9},
 {'well': '8н', 'top': 1792.8, 'bot': 1796.2, 'soil': 81.2},
 {'well': '8н', 'top': 1797.2, 'bot': 1802.8, 'soil': 64.1},
 {'well': '9н', 'top': 1770.6,

In [ ]:
def bisect_left(a, x, lo=0, hi=None, param='bot'):
    if lo < 0:
        raise ValueError('lo must be non-negative')
    if hi is None:
        hi = len(a)
    while lo < hi:
        mid = (lo + hi) // 2
        if a[mid][param] < x:
            lo = mid + 1
        else:
            hi = mid
    return lo

def get_actual_perf(perf_ints, act_perf_year=None):
    act_perf = []
    if act_perf_year is None:
        act_perf_year = datetime.datetime.now().date()
    for well in tqdm(perf_ints.keys()):
        act_perf_well = []
        for row in perf_ints[well]:
            if row['date'] > act_perf_year:
                continue
            top = row['top']
            bot = row['bot']
            perf_type = row['type']

            idx = bisect_left(act_perf_well, top)
            if idx == len(act_perf_well):
                act_perf_well.append({'well': well, 'top': top, 'bot': bot,
                                      'perf_type': perf_type})
            else:
                shift = 1
                if act_perf_well[idx]['top'] > top:
                    if act_perf_well[idx]['perf_type'] == perf_type:
                        act_perf_well[idx]['top'] = top
                    else:
                        act_perf_well.insert(idx,
                                             {'well': well, 'top': top,
                                              'bot': bot if
                                              act_perf_well[idx][
                                                  'top'] > bot else
                                              act_perf_well[idx]['top'],
                                              'perf_type': perf_type})
                        shift += 1
                if act_perf_well[idx]['bot'] < bot:
                    if act_perf_well[idx]['perf_type'] == perf_type:
                        act_perf_well[idx]['bot'] = bot
                    else:
                        act_perf_well.insert(idx + shift,
                                             {'well': well,
                                              'top': act_perf_well[idx]['bot'],
                                              'bot': bot,
                                              'perf_type': perf_type})

        act_perf.extend(act_perf_well)
    return act_perf

In [ ]:
perf = get_actual_perf(perf_ints)
